In [1]:
# импортируем библиотеки numpy и pandas
import numpy as np
import pandas as pd

# импортируем библиотеку datatime для работы с датами
import datetime
from datetime import datetime, date

# задаем некоторые опции библиотеки pandas, которые настраивают вывод
pd.set_option('display.notebook_repr_html', False)     # задаем вывод в виде текста, а не HTML
pd.set_option('display.max_columns', 8)                # устанавливаем отображение максимального количества стобцов
pd.set_option('display.max_rows', 10)                  # устанавливаем отображение максимального количества строк
pd.set_option('display.width', 80)                     # устанавливаеv максимальную ширину отображения в символах

# импортируем библиотеку matplotlib для построения графиков
import matplotlib.pyplot as plt 
%matplotlib inline

In [2]:
# считываем из файла данные и выводим первые 5 строк
sp500 = pd.read_csv("./sp500.csv",
                   index_col = 'Symbol',     # используем столбец Symbol в качестве индекса
                   usecols = [0, 2, 3, 7])   # и считываем только те столбцы у которых позиции 0, 2, 3, 7
sp500.head()

                        Sector   Price  Book Value
Symbol                                            
MMM                Industrials  141.14      26.668
ABT                Health Care   39.60      15.573
ABBV               Health Care   53.95       2.954
ACN     Information Technology   79.79       8.326
ACE                 Financials  102.91      86.897

In [3]:
# создаем DataFrame, состоящий из случайных чисел и столбца key 
np.random.seed(123456)
df = pd.DataFrame({'foo':np.random.random(10000), 'key':range(100, 10100)})
df[:5]

        foo  key
0  0.126970  100
1  0.966718  101
2  0.260476  102
3  0.897237  103
4  0.376750  104

In [4]:
# отбираем строку, в которой значение столбца key равно 10 099
df[df.key==10099]

           foo    key
9999  0.272283  10099

In [5]:
# измеряем время выполнения операции отбора
%time df[df.key==10099]

CPU times: total: 0 ns
Wall time: 994 µs


           foo    key
9999  0.272283  10099

In [6]:
# превращаем столбец key в index
df_with_index = df.set_index(['key'])
df_with_index[:5]

          foo
key          
100  0.126970
101  0.966718
102  0.260476
103  0.897237
104  0.376750

In [7]:
# теперь можно найти это значение с помощью свойства .loc[]
df_with_index.loc[10099]

foo    0.272283
Name: 10099, dtype: float64

In [8]:
%time df_with_index.loc[10099]

CPU times: total: 0 ns
Wall time: 0 ns


foo    0.272283
Name: 10099, dtype: float64

In [9]:
# операция с помощью индекса выполняется примерно в пять раз быстрее. В силу лучшей производительности поиска по индексу
# (в тех случаях когда это возможно) обычно является оптимальным решением. Недостаток использования индекса заключается в том,
# что потребуется время на его создание и он занимает больше памяти

__Типы индексов библиотеки Pandas__

__Основной тип Index__ - представляет собой упорядоченный набор значений, на основе которого можно сделать срез. Значения которые он содержит, должны быть хешируемыми питоновскими объектами.

In [10]:
# Покажем, что столбцы фактически являются индексом
temps = pd.DataFrame({'City': ['Missoula', 'Philadelphia'], 'Temperature': ['70', '80']})
temps

           City Temperature
0      Missoula          70
1  Philadelphia          80

In [11]:
# мы видим, что столбцы - это индекс
temps.columns

Index(['City', 'Temperature'], dtype='object')

__Индексы int64index и RangeIndex, в качестве меток используются целые числа__

__int64index__ - представляет собой иммутабельный (неизменяемый) массив, состоящий из 64-битовых целых чисел, которые сопоставляются значениям.

In [12]:
# явно создаем int64index
df_i64 = pd.DataFrame(np.arange(10, 20), index = np.arange(0, 10))
df_i64[:5]

    0
0  10
1  11
2  12
3  13
4  14

In [13]:
# смотрим index
df_i64.index

Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype='int64')

__Rangeindex__


In [14]:
# по умолчание используется Rangeindex
df_range = pd.DataFrame(np.arange(10, 15))
df_range

    0
0  10
1  11
2  12
3  13
4  14

In [15]:
df_range.index

RangeIndex(start=0, stop=5, step=1)

__Индекс Float64index__, в качестве меток использует числа с плавающей точкой

In [16]:
# индексы использующие Float64index
df_f64 = pd.DataFrame(np.arange(0, 1000, 5), np.arange(0.0, 100.0, 0.5))
df_f64.iloc[:5] # потребуется .iloc [] для отбора пяти первых строк

      0
0.0   0
0.5   5
1.0  10
1.5  15
2.0  20

In [17]:
df_f64.index

Float64Index([ 0.0,  0.5,  1.0,  1.5,  2.0,  2.5,  3.0,  3.5,  4.0,  4.5,
              ...
              95.0, 95.5, 96.0, 96.5, 97.0, 97.5, 98.0, 98.5, 99.0, 99.5],
             dtype='float64', length=200)

__intervalindex__ представление дискретных интервалов

In [18]:
# создаем датафрейм с использованием intervalindex
df_interval = pd.DataFrame({'A': [1, 2, 3, 4]}, 
                           index = pd.IntervalIndex.from_breaks([0, 0.5, 1.0, 1.5, 2.0]))
df_interval

            A
(0.0, 0.5]  1
(0.5, 1.0]  2
(1.0, 1.5]  3
(1.5, 2.0]  4

In [19]:
df_interval.index

IntervalIndex([(0.0, 0.5], (0.5, 1.0], (1.0, 1.5], (1.5, 2.0]], dtype='interval[float64, right]')

__CategoricalIndex__ - категории в качестве индекса

In [20]:
# создаем датафрейм со столбцом имеющим тип Categorical 
df_categorical = pd.DataFrame({'A': np.arange(6),
                               'B': list('aabbca')})
# импортируем класс CategoricalDtype для работы с категориальными данными
from pandas.api.types import CategoricalDtype
df_categorical['B'] = df_categorical['B'].astype(CategoricalDtype(categories = list('cab')))
df_categorical

   A  B
0  0  a
1  1  a
2  2  b
3  3  b
4  4  c
5  5  a

In [21]:
# превращаем категориальный столбец в индекс
df_categorical = df_categorical.set_index('B')
df_categorical.index

CategoricalIndex(['a', 'a', 'b', 'b', 'c', 'a'], categories=['c', 'a', 'b'], ordered=False, dtype='category', name='B')

In [22]:
# ищем значения относящиеся к категории 'a'
df_categorical.loc['a']

   A
B   
a  0
a  1
a  5

__Datetimeindex__ - индексация по датам и времени

In [23]:
# создаем Datetimeindex на основе диапазона дат
rng = pd.date_range('5/1/2017', periods = 5, freq = 'H')
ts = pd.Series(np.random.randn(len(rng)), index = rng)
ts

2017-05-01 00:00:00    1.239792
2017-05-01 01:00:00   -0.400611
2017-05-01 02:00:00    0.718247
2017-05-01 03:00:00    0.430499
2017-05-01 04:00:00    1.155432
Freq: H, dtype: float64

In [24]:
ts.index

DatetimeIndex(['2017-05-01 00:00:00', '2017-05-01 01:00:00',
               '2017-05-01 02:00:00', '2017-05-01 03:00:00',
               '2017-05-01 04:00:00'],
              dtype='datetime64[ns]', freq='H')

__Periodindex__ - индексация периодов времени

In [25]:
# явно создаем PeruodIndex
periods = pd.PeriodIndex(['2017-1', '2017-2', '2017-3'], freq = 'M')
periods

PeriodIndex(['2017-01', '2017-02', '2017-03'], dtype='period[M]')

In [26]:
# используем индекс в серии
period_series  = pd.Series(np.random.randn(len(periods)), index = periods)
period_series

2017-01   -0.449276
2017-02    2.472977
2017-03   -0.716023
Freq: M, dtype: float64

# __Работа с индексами__

__Создание и использование индекса в объекте Series или объекте DataFrame__

In [27]:
# индексы можно создавать либо явно, либо позволить библиотеке pandas создать их не явно. Явное создание индекса
# происходит когда задается индекс используя параметр index конструктора

# создаем DatetimeIndex
date_time = pd.DatetimeIndex(pd.date_range('5/1/2017', periods = 5, freq = 'H'))
date_time

DatetimeIndex(['2017-05-01 00:00:00', '2017-05-01 01:00:00',
               '2017-05-01 02:00:00', '2017-05-01 03:00:00',
               '2017-05-01 04:00:00'],
              dtype='datetime64[ns]', freq='H')

In [28]:
# создаем объект DataFrame, используя индекс
df_date_times = pd.DataFrame(np.arange(0, len(date_time)), index = date_time)
df_date_times

                     0
2017-05-01 00:00:00  0
2017-05-01 01:00:00  1
2017-05-01 02:00:00  2
2017-05-01 03:00:00  3
2017-05-01 04:00:00  4

In [29]:
# задаем индекс датафрейма непосредственно
df_date_times.index = pd.DatetimeIndex(pd.date_range('6/1/2017', periods = 5, freq = 'H'))
df_date_times

                     0
2017-06-01 00:00:00  0
2017-06-01 01:00:00  1
2017-06-01 02:00:00  2
2017-06-01 03:00:00  3
2017-06-01 04:00:00  4

__Отбор значений с помощью индекса__

Поиск в объекте Series можно выполнить с помощью:
- __.loc[ ]__ - выполняет поиск по метке а не по позиции. Однако будьте осторожны: если метки являются целыми числами,
то целые числа будут обрабатываться как метки
- __.at[ ]__ - работает как .loc, но при этом может извлечь только одно значение
- __.iloc[ ]__ - выполняет поиск по позиции, начинающейся с 0, а не по метке
- __[ ]__ - извлекает значение для указанной метки

In [30]:
# создаем серию
s = pd.Series(np.arange(0, 5), index = list('abcde'))
s

a    0
b    1
c    2
d    3
e    4
dtype: int32

In [31]:
# ищем по метке индекса
s['b']

1

In [32]:
# явно ищем по метке индекса
s.loc['b']

1

In [33]:
# когда оператор [ ] применяется к датафрейму, он извлекает столбцы вместо строк
# создаем датафрейм с двумя столбцами
df = pd.DataFrame([np.arange(10, 12),
                  np.arange(12, 14)],
                  columns = list('ab'),
                  index = list('vw'))
df

    a   b
v  10  11
w  12  13

In [34]:
df['a']

v    10
w    12
Name: a, dtype: int32

In [35]:
# чтобы выполнить поиск строки внутри DataFrame необходимо воспользоваться одним из индексаторов свойств
df.loc['w']

a    12
b    13
Name: w, dtype: int32

In [36]:
# кроме того, индексаторы-свойства могут использовать срезы
# создаем срез серии от метки b до метки d
s['b':'d']

b    1
c    2
d    3
dtype: int32

In [37]:
# эта строка явно создает срез серии от метки b до метки d
s.loc['b':'d']

b    1
c    2
d    3
dtype: int32

In [38]:
# кроме того можно передать список значений
s.loc[['a', 'c', 'e']]

a    0
c    2
e    4
dtype: int32

__Преобразование данных в индекс и получение данных из индекса__

In [39]:
# с помощью метода .reset_index() можно сбросить индекс датафрейма. Следующий програмный код помещает символы 
# в индексе датафрейма sp500 в столбец и заменяет индекс стандартным целочисленным индексом
# исследуем несколь строк датафрейма sp500
sp500[:5]

                        Sector   Price  Book Value
Symbol                                            
MMM                Industrials  141.14      26.668
ABT                Health Care   39.60      15.573
ABBV               Health Care   53.95       2.954
ACN     Information Technology   79.79       8.326
ACE                 Financials  102.91      86.897

In [40]:
# сбрасываем индекс, помещая значение индекса в столбец
index_moved_to_col = sp500.reset_index()
index_moved_to_col[:5]

  Symbol                  Sector   Price  Book Value
0    MMM             Industrials  141.14      26.668
1    ABT             Health Care   39.60      15.573
2   ABBV             Health Care   53.95       2.954
3    ACN  Information Technology   79.79       8.326
4    ACE              Financials  102.91      86.897

In [41]:
# Столбец данных можно поместить в индекс объектф DataFrame, возпользовавшись методом .set_index() и указав перемещаемый столбец
# делаем столбец Sector индексом 
index_moved_to_col.set_index('Sector')[:5]

                       Symbol   Price  Book Value
Sector                                           
Industrials               MMM  141.14      26.668
Health Care               ABT   39.60      15.573
Health Care              ABBV   53.95       2.954
Information Technology    ACN   79.79       8.326
Financials                ACE  102.91      86.897

__Переиндексация объекта библиотеки pandas__

In [42]:
# Переиндексация обеспечит совместимость датафрейма с новым индексом, выровняв данные относящиеся к старому индексу, по 
# новому индексу и встави значения Nan там, где невозможно выполнить выравнивание
# делаем переиндексацию задав метки MMM, ABBV, FOO
reindexed = sp500.reindex(index = ['MMM', 'ABBV', 'FOO'])
reindexed
# эта операция создает новый объект DataFrame

             Sector   Price  Book Value
Symbol                                 
MMM     Industrials  141.14      26.668
ABBV    Health Care   53.95       2.954
FOO             NaN     NaN         NaN

In [43]:
# выполняем переиндексацию столбцов
sp500.reindex(columns=['Price', 'Book Value', 'NewCol'])[:5]

         Price  Book Value  NewCol
Symbol                            
MMM     141.14      26.668     NaN
ABT      39.60      15.573     NaN
ABBV     53.95       2.954     NaN
ACN      79.79       8.326     NaN
ACE     102.91      86.897     NaN

__Иерархическая индексация__

In [44]:
# мы хотим организовать данные по значением столбцов Sector и Symbol, что бы в дальнейшем осуществлять эффективный поиск
# информации на основе комбинаций значений обеих переменных
# сначала помещаем символы в столбец
reindexed = sp500.reset_index()
# а теперь проиндекисруем DataFrame по столбцац Sector и Symbol 
multi_fi = reindexed.set_index(['Sector', 'Symbol'])
multi_fi.head()

                                Price  Book Value
Sector                 Symbol                    
Industrials            MMM     141.14      26.668
Health Care            ABT      39.60      15.573
                       ABBV     53.95       2.954
Information Technology ACN      79.79       8.326
Financials             ACE     102.91      86.897

In [45]:
# Наш индекс это MultiIndex
type(multi_fi.index)

pandas.core.indexes.multi.MultiIndex

In [46]:
len(multi_fi.index.levels)

2

In [47]:
# каждый уровень идкса - это индекс
multi_fi.index.levels[0]

Index(['Consumer Discretionary', 'Consumer Discretionary ', 'Consumer Staples',
       'Consumer Staples ', 'Energy', 'Financials', 'Health Care',
       'Industrials', 'Industries', 'Information Technology', 'Materials',
       'Telecommunications Services', 'Utilities'],
      dtype='object', name='Sector')

In [48]:
# каждый уровень идкса - это индекс
multi_fi.index.levels[1]

Index(['A', 'AA', 'AAPL', 'ABBV', 'ABC', 'ABT', 'ACE', 'ACN', 'ACT', 'ADBE',
       ...
       'XLNX', 'XOM', 'XRAY', 'XRX', 'XYL', 'YHOO', 'YUM', 'ZION', 'ZMH',
       'ZTS'],
      dtype='object', name='Symbol', length=500)

In [49]:
# Значения самого индекса в определенном уровне для каждой строки можно извлеч с помощью метода .get_level_values()
multi_fi.index.get_level_values(0)

Index(['Industrials', 'Health Care', 'Health Care', 'Information Technology',
       'Financials', 'Health Care', 'Information Technology', 'Utilities',
       'Health Care', 'Financials',
       ...
       'Utilities', 'Information Technology', 'Information Technology',
       'Financials', 'Industrials', 'Information Technology',
       'Consumer Discretionary', 'Health Care', 'Financials', 'Health Care'],
      dtype='object', name='Sector', length=500)

In [50]:
# Чтобы посмотреть значения DataFrame  с помощью иерархического индекса необходимо воспользоваться методом .xs()
# получаем все акции, которые имеют значение Indusrials
# обратите внимание, что в результатах индекс уровня 0 не выводится
multi_fi.xs('Industrials')[:5]

# индекс полученного датафрейма состоит из уровней которые не были заданы, в данном случае это Symbol 
# Уровни для которых были указаны значения, удаляются из полученного индекса

         Price  Book Value
Symbol                    
MMM     141.14      26.668
ALLE     52.46       0.000
APH      95.71      18.315
AVY      48.20      15.616
BA      132.41      19.870

In [51]:
# отбираем строки, в которых индекс уровня 1 (Symbol) имеет значение ALLE
# обратите внимание что в результатах индекс уровня 1 Symbol не выводится
multi_fi.xs('ALLE', level = 1)

             Price  Book Value
Sector                        
Industrials  52.46         0.0

In [52]:
# что бы предотвратить удаление уровней из результатов, можно воспользоваться параметром drop_levels=False
# отбираем строки, в которых индекс уровня 0 Sector имеет занчение indastrial, без удаления уровней
multi_fi.xs('Industrials', drop_level=False)[:5]

                     Price  Book Value
Sector      Symbol                    
Industrials MMM     141.14      26.668
            ALLE     52.46       0.000
            APH      95.71      18.315
            AVY      48.20      15.616
            BA      132.41      19.870

In [53]:
# что бы отобрать строку используя иерархию индексов, вы можете связать вместе вызовы .xs() для разных уровней
# скомбинируем уровни индексов
multi_fi.xs('Industrials').xs('UPS')

Price         102.73
Book Value      6.79
Name: UPS, dtype: float64

In [54]:
# как вариант можно передать значения каждого уровня в виде кортежа
multi_fi.xs(('Industrials', 'UPS'))

Price         102.73
Book Value      6.79
Name: (Industrials, UPS), dtype: float64